In [1]:
import tensorflow as tf
from tensorflow import keras
from loguru import logger
from tensorflow.keras.models import Model

2022-11-03 19:26:08.590696: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
train = "/home/denmann99/autoencoder/pics/denver1"
denver=tf.keras.preprocessing.image_dataset_from_directory(train,validation_split=0.2,subset='training',shuffle=True,
                                                     batch_size=64,image_size=(256, 256),seed=123)
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

def change_inputs(images, labels):
  x = tf.image.resize(normalization_layer(images),[256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return x, x

normalized_denver = denver.map(change_inputs)

Found 903 files belonging to 1 classes.
Using 723 files for training.


2022-11-03 19:26:10.288639: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-03 19:26:10.292966: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-03 19:26:10.330201: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 19:26:10.330441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 245.91GiB/s
2022-11-03 19:26:10.330460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-03 19:26:10.351894: I tensorflow/stream_executor/platform/def

In [3]:
input = tf.keras.layers.Input(shape=(256, 256, 3))

# Encoder
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 128, 128, 32)      9248      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 256, 256, 32)      9248  

In [4]:
autoencoder.fit(
    x=normalized_denver,
    epochs=50,
    batch_size=64,
    shuffle=True
)

Epoch 1/50


2022-11-03 19:26:11.296344: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-03 19:26:11.313791: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599990000 Hz
2022-11-03 19:26:13.177751: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-11-03 19:26:14.233527: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-11-03 19:26:14.305319: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-11-03 19:26:15.437701: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-11-03 19:26:15.509829: E tensorflow/stream

 1/12 [=>............................] - ETA: 3:23 - loss: 0.6936

2022-11-03 19:26:29.814459: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED


11/12 [==========================>...] - ETA: 1s - loss: 0.6868

2022-11-03 19:26:44.087572: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-11-03 19:26:45.470698: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-11-03 19:26:46.049921: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-11-03 19:26:46.772850: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-11-03 19:26:48.042350: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-11-03 19:26:48.989037: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-11-03 19:26:49.760036: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-1